In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.activations import relu
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import ResNet50

In [ ]:

!pip install kaggle

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid
!unzip -q monkeypox-skin-images-dataset-msid

 77% 42.0M/54.6M [00:00<00:00, 158MB/s]
100% 54.6M/54.6M [00:00<00:00, 114MB/s]


In [6]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [7]:
baseModel = ResNet50(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(4, activation='softmax')(dense2)


94765736/94765736 [==============================] - 1s 0us/step


In [8]:
model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 75100804 (286.49 MB)
Trainable params: 51513092

In [10]:
history = model.fit(
    TrainData,
    epochs=50,  # adjust the number of epochs as needed
    validation_data=validationData
)

Epoch 1/50
20/20 [==============================] - 16s 358ms/step - loss: 5.8907 - accuracy: 0.3754 - val_loss: 2.6089 - val_accuracy: 0.1513
Epoch 2/50
20/20 [==============================] - 4s 187ms/step - loss: 2.2185 - accuracy: 0.3997 - val_loss: 1.7129 - val_accuracy: 0.2763
Epoch 3/50
20/20 [==============================] - 3s 149ms/step - loss: 1.4930 - accuracy: 0.4725 - val_loss: 1.2197 - val_accuracy: 0.5526
Epoch 4/50
20/20 [==============================] - 3s 150ms/step - loss: 1.1189 - accuracy: 0.5631 - val_loss: 1.7331 - val_accuracy: 0.2566
Epoch 5/50
20/20 [==============================] - 3s 150ms/step - loss: 1.4031 - accuracy: 0.4935 - val_loss: 1.3968 - val_accuracy: 0.3553
Epoch 6/50
20/20 [==============================] - 4s 183ms/step - loss: 1.0620 - accuracy: 0.5388 - val_loss: 1.1797 - val_accuracy: 0.5855
Epoch 7/50
20/20 [==============================] - 3s 149ms/step - loss: 1.1200 - accuracy: 0.5518 - val_loss: 1.8961 - val_accuracy: 0.2171
Epoch

In [11]:
model.save("ResNet50.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
